In [1]:
# 準備工作

from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import datetime
import re
import sqlite3
import os
import json
import pprint

db_name = "../data/21_century.sqlite"

In [2]:
# 從db_name取出資料

with sqlite3.connect(db_name) as conn_retrieve:
    
    c_rt = conn_retrieve.cursor()
    qryString = "SELECT * from rental;"
    c_rt.execute(qryString)
    retrieved_list = c_rt.fetchall()

c_rt.close()

In [3]:
# 定義 recover_quot() 方法 (recover quotation marks)
# 將字串中連續兩個的單引號跟雙引號復原為
# 單一的單引號跟雙引號

def recover_quot(string):
    string = string.replace("\"\"","\"")
    string = string.replace("\'\'","\'")
    return string

In [4]:
# 定義 remove_dust() 方法
# 清理一個字串中所有的 \r, \n, \t, \xa0 符號
# 並將頭尾的空格去掉

def remove_dust(string):
    string = string.replace("\r","")
    string = string.replace("\n","")
    string = string.replace("\t","")
    string = string.replace("\xa0","")
    string = string.strip()
    return string

In [5]:
# 定義 get_section_data() 方法
# 從每個章節中抽出資料

def get_section_data():
    
    section_data = {}
    sections = [each.text for each in soup.find_all('h3')]
    
    for section_name in sections:

        section = soup.find_all('h3',text=section_name)[0]
        section = section.find_parent()

        if not section_name == "環境介紹":

            section_key_tags = section.select('h6')
            section_keys = [each.text for each in section_key_tags]
            section_values = [each.find_next().text for each in section_key_tags]

        else:

            section_keys = ['環境介紹']
            section_values = [section.select('div > div')[0].text]
        
        
        # 清理不必要的字串
        
        section_keys = [remove_dust(each) for each in section_keys]
        section_values = [remove_dust(each) for each in section_values]
        
        
        # 塞值到 section_dict 中

        section_data.update(dict(zip(section_keys, section_values)))   
    
    # 回傳
    
    return section_data

In [90]:
# 建立 get_data_dict 方法
# 給代表網頁內容的soup物件
# 取得每頁的資料 並以dictionary形式傳回

def get_data_dict(soup):

    # 內頁標籤1(如"租屋")
    tag01 = soup.select('div[id="breadCrumb"] > ul > li')[1].text

    # 內頁標籤2(如"租屋列表")
    tag02 = soup.select('div[id="breadCrumb"] > ul > li')[2].text

    # 內頁標籤3(如"光復南路優質地段B1(租10)")
    tag03 = soup.select('div[id="breadCrumb"] > ul > li')[3].text


    # 仲介資訊
    
    try:
 
        agent_tags = soup.find_all('div',class_=re.compile("column staff"))
        agent_names = [each.select_one('h4').text for each in agent_tags]

        # 證號

        agent_licenses = [remove_dust(each.find('div',class_="LicenseNo").text) for each in agent_tags]

        # 電話

        agent_contact_tags = soup.select('div[name="agentContactInfo"]')
        agent_phones = []

        # 先找出每個仲介的所有電話(可能不只1個) 個別存成list
        # 但因為select後每個仍為含有tag的list
        # 所以得再將每個list裡面的tag挑出來 取出真正的電話資料後再存回

        for each_list in agent_contact_tags:
            each_agent_phones = []
            each_list = each_list.select('span[class="mobileInfo"] > a')

            for each in each_list:
                each_agent_phones.append(each.text)

            agent_phones.append(each_agent_phones)


        # email
        # 跟類似處理電話的方式處理

        agent_emails_tags = [each.find_all('a',href=re.compile("mailto:")) for each in agent_contact_tags]
        agent_emails = []

        for each_list in agent_emails_tags:
            each_agent_emails = []  

            for each in each_list:
                each_agent_emails.append(each.text)

            agent_emails.append(each_agent_emails)




        # 整合仲介資料

        agent_data = {}

        # each_agent_data = {
        #     '證號': agent_licenses,
        #     '電話': agent_phones,
        #     'email': agent_emails        
        # }

        for j in range(0,len(agent_names)):

            agent_data.update(
                {agent_names[j]: {    # 代表仲介姓名
                    '證號': agent_licenses[j],
                    '電話': agent_phones[j],
                    'email': agent_emails[j]  
                }
            })

    except:
        
        agent_data = "NULL"
    
        
    # 房東資訊

    try:
        
        section = soup.find('div',class_=re.compile("column owner"))
        landlord_name = section.select_one('h3').text
        landlord_details = section.select_one('span').text

        landlord_data = {
            'name': landlord_name,
            'details': landlord_details
        }
    
    except:
        
        landlord_data = "NULL"
    

    # 加盟店

    try:

        branch_tags = soup.find('div',class_=re.compile("column store"))
        branch_name = branch_tags.select_one('h4').text
        branch_co_name = branch_tags.select('ul > li')[0].text
        branch_address = branch_tags.select('ul > li')[1].text
        branch_phone = branch_tags.select('ul > li')[2].text
        branch_fax = branch_tags.select('ul > li')[3].text

        branch_data = {
            '名稱': branch_address,
            '分行公司': branch_co_name,  
            '地址': branch_address,
            '電話': branch_phone,
            '傳真': branch_fax
        }

        # 整理加盟店資訊

        branch_data = {key:remove_dust(branch_dict[key]).strip() for key in store_dict }
        
    except:
        
        branch_data = "NULL"
    
    




    # 存放單筆地點資料用的dictionary
    data_dict = {
        '標籤01': tag01,
        '標籤02': tag02,
        '地點名稱': tag03,
        '仲介': agent_data,
        '房東': landlord_data,
        '分行': branch_data
    }


    # 跟其他dictionary合併

    data_dict.update(get_section_data())


    # 傳回結果

    return data_dict

In [91]:
# 跑迴圈 清洗資料
# 並將結果暫存到一個大的dictionary中

# index_set = list(range(0,len(retrieved_list)))
index_set = range(0,10)    # 測試用
dict_all = {}

for i in index_set:

    # 將內頁內容轉成 BeautifulSoup 物件

    page_source = retrieved_list[i][1]
    page_source = recover_quot(page_source) # 還原引號
    soup = BeautifulSoup(page_source,'html.parser')

    # 網址

    url = retrieved_list[i][0]
    posting_id = url.split("/")[-1]

    print(i)
    print(posting_id)
    
    # 清理

    data_dict = get_data_dict(soup)
    data_dict.update({'url':url})

    pprint.pprint(data_dict)

    # 暫存

    dict_all.update({posting_id: data_dict})

pprint.pprint(dict_all)

0
10669
{'url': 'http://www.century21.com.tw/index/Rental/RentPage/10669',
 '仲介': {},
 '其他說明': '可養寵物、不與房東同住',
 '分行': 'NULL',
 '地址': '台北市中山區建國北路三段',
 '地點名稱': '濱江陽光雅寓住家~2大套房',
 '坪數': '32坪',
 '性別限制': '男女皆可',
 '房東': {'details': '方便聯絡時間：09:00~12:00 或 14:00~22:00', 'name': '藍'},
 '押金': '2個月',
 '樓層': '5/5樓',
 '標籤01': '租屋',
 '標籤02': '租屋列表',
 '環境介紹': '',
 '租金': '每月30,000元',
 '種類': '公寓'}
1
11876
{'url': 'http://www.century21.com.tw/index/Rental/RentPage/11876',
 '仲介': {'陳秀吟': {'email': ['show483055@yahoo.com.tw'],
                '證號': '營業員證號：(104)登字第295492號',
                '電話': ['0933075795']}},
 '其他說明': '不與房東同住',
 '分行': 'NULL',
 '地址': '新北市三芝區田心子',
 '地點名稱': '田心子廠房出租',
 '坪數': '250坪',
 '性別限制': '男女皆可',
 '房東': 'NULL',
 '押金': '面議',
 '樓層': '1-2',
 '標籤01': '租屋',
 '標籤02': '租屋列表',
 '環境介紹': '1~2樓可分租 共約250坪；租金誠可談；有空地停車方便；距離北新路100公尺',
 '租金': '每月120,000元',
 '種類': '廠房'}
2
13050
{'url': 'http://www.century21.com.tw/index/Rental/RentPage/13050',
 '交通狀況': '三民高中捷運',
 '仲介': {},
 '其他說明': '可養寵物、不與房東同住',
 '分行': '

In [89]:
i = 8

page_source = retrieved_list[i][1]
page_source = recover_quot(page_source) # 還原引號
soup = BeautifulSoup(page_source,'html.parser')


{'賴燕樺': {'email': ['j197@c21.tw'],
         '證號': '營業員證號：(96)登字第091002號',
         '電話': ['0912786705']},
 '陳健樺': {'email': ['dawson.jack@msa.hinet.net'],
         '證號': '營業員證號：(95)登字第068430號',
         '電話': ['0970705093']}}
